In [12]:
!pip install faiss-cpu gradio transformers soundfile

In [13]:
import os, shutil
import faiss, json
import numpy as np
import torch, tempfile
import IPython.display as ipd
import threading, time
import requests
import gradio as gr
import soundfile as sf 
import io, glob # Import glob to list files in a directory
from PIL import Image
from io import BytesIO
from IPython.display import Audio # Keep for potential debugging/display outside Gradio
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizerFast
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

In [ ]:
# --- Configuration ---
# Assuming the index and metadata files are already generated and saved from a previous run
# If not, the indexing pipeline code would need to be included here as well.
INDEX_FILE = "image_search.index"
METADATA_FILE = "image_search_metadata.json"
IMAGE_DIR = "data_images" # Directory where images were indexed from
MODEL_ID = "openai/clip-vit-base-patch32"
HF_API_TOKEN = "ADD_YOUR_TOKEN" # Define the HF API token
headers = {"Authorization": f"Bearer {HF_API_TOKEN}","Content-Type": "audio/wav"} # Define headers for HF API

In [15]:
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)
    print(f"Created directory: {IMAGE_DIR}")

# --- Upload your images to the '{IMAGE_DIR}' directory for indexing. ---
print(f"\n--- Upload Images for Indexing (to '{IMAGE_DIR}') ---")


--- Upload Images for Indexing (to 'data_images') ---


In [16]:
# 1. Source (Kaggle dataset path)
source_image_directory = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/'
# --- Setup: Create Destination Directory ---
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)
    print(f"Created destination directory: {IMAGE_DIR}")

# --- Copy Images from Source to Destination ---
print(f"Starting to copy images from {source_image_directory} to {IMAGE_DIR}...")

copied_count = 0
total_files_in_source = 0

# os.walk traverses the source directory structure
for dirname, _, filenames in os.walk(source_image_directory):
    total_files_in_source += len(filenames)
    
    # Iterate over files found in the current subdirectory (dirname)
    for filename in filenames:
        
        # 1. Construct the full path of the source file
        src_path = os.path.join(dirname, filename)
        
        # 2. Construct the full path of the destination file
        dst_path = os.path.join(IMAGE_DIR, filename)
        
        try:
            # 3. Use shutil.copy to copy the file
            # This is the line that actually "uploads" (copies) the image to your working folder.
            shutil.copy(src_path, dst_path)
            copied_count += 1
        except Exception as e:
            print(f"Warning: Could not copy file {filename}: {e}")
            
    # Optional: If you only want to copy a small subset for quick testing, 
    # you can uncomment and adjust this break condition:
    # if copied_count > 1000:
    #     print("Copied 1000 images, stopping early for test.")
    #     break

# --- Summary ---
print(f"\nFinished copying. Total files found in source: {total_files_in_source}")
print(f"Successfully copied {copied_count} files to {IMAGE_DIR}.")

# You can now proceed with your indexing pipeline using IMAGE_DIR

Starting to copy images from /kaggle/input/fashion-product-images-dataset/fashion-dataset/images/ to data_images...

Finished copying. Total files found in source: 44441
Successfully copied 44441 files to data_images.


In [17]:
# --- Setup ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load Model and Processor once
try:
    model = CLIPModel.from_pretrained(MODEL_ID).to(device)
    processor = CLIPProcessor.from_pretrained(MODEL_ID)
    tokenizer = CLIPTokenizerFast.from_pretrained(MODEL_ID) # Load tokenizer for text
    print("CLIP model, processor, and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading CLIP model or processor: {e}")
    model, processor, tokenizer = None, None, None # Set to None if loading fails

Using device: cuda
CLIP model, processor, and tokenizer loaded successfully.


In [18]:
 #--- Indexing Pipeline ---

def get_image_embedding(image_path: str, model, processor, device) -> np.ndarray:
    """Generates the CLIP embedding vector for a single image file path."""
    if model is None or processor is None:
        print("Error: CLIP model or processor not loaded.")
        return None
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            image_features = model.get_image_features(pixel_values=inputs['pixel_values'])

        embedding = image_features.cpu().numpy().flatten()
        # Normalize the embedding
        embedding = embedding / np.linalg.norm(embedding)
        return embedding.astype('float32')
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

def create_image_index(image_dir: str, index_file: str, metadata_file: str, model, processor, device):
    """
    Creates a FAISS index and metadata file for images in a directory.
    """
    if model is None or processor is None:
        print("Error: CLIP model or processor not loaded, cannot create index.")
        return

    image_paths = glob.glob(os.path.join(image_dir, '*')) # Get all files in the image directory
    image_paths = [f for f in image_paths if os.path.isfile(f) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))] # Filter for image files

    if not image_paths:
        print(f"No image files found in directory: {image_dir}. Cannot create index.")
        # Removed the repeated ACTION REQUIRED message here as upload is now handled above
        return

    embeddings_list = []
    valid_image_paths = [] # Store paths of images that were successfully processed

    print(f"Found {len(image_paths)} images. Generating embeddings...")
    for i, image_path in enumerate(image_paths):
        embedding = get_image_embedding(image_path, model, processor, device)
        if embedding is not None:
            embeddings_list.append(embedding)
            valid_image_paths.append(image_path) # Only add path if embedding was successful
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1}/{len(image_paths)} images.")

    if not embeddings_list:
        print("No embeddings generated. Cannot create index.")
        return

    embeddings_matrix = np.vstack(embeddings_list)
    dimension = embeddings_matrix.shape[1]

    print(f"Creating FAISS index with dimension {dimension}...")
    index = faiss.IndexFlatIP(dimension) # Use Inner Product for normalized embeddings
    index.add(embeddings_matrix)

    print(f"Index created with {index.ntotal} vectors.")

    # Save the index and metadata
    faiss.write_index(index, index_file)
    with open(metadata_file, 'w') as f:
        json.dump(valid_image_paths, f) # Save the valid image paths

    print(f"FAISS index saved to {index_file}")
    print(f"Image paths metadata saved to {metadata_file}")

In [19]:
if not os.path.exists(INDEX_FILE) or not os.path.exists(METADATA_FILE):
    print("Index files not found. Creating index...")
    # Ensure IMAGE_DIR exists and contains images before calling create_image_index
    if os.path.exists(IMAGE_DIR):
        create_image_index(IMAGE_DIR, INDEX_FILE, METADATA_FILE, model, processor, device)
    else:
        # This case should ideally not happen if the directory creation above was successful
        print(f"Error: Image directory '{IMAGE_DIR}' not found. Cannot create index.")
        print("Please upload your image data to the specified directory.")
else:
    print("Index files found. Skipping index creation.")

Index files not found. Creating index...
Found 44441 images. Generating embeddings...
Processed 100/44441 images.
Processed 200/44441 images.
Processed 300/44441 images.
Processed 400/44441 images.
Processed 500/44441 images.
Processed 600/44441 images.
Processed 700/44441 images.
Processed 800/44441 images.
Processed 900/44441 images.
Processed 1000/44441 images.
Processed 1100/44441 images.
Processed 1200/44441 images.
Processed 1300/44441 images.
Processed 1400/44441 images.
Processed 1500/44441 images.
Processed 1600/44441 images.
Processed 1700/44441 images.
Processed 1800/44441 images.
Processed 1900/44441 images.
Processed 2000/44441 images.
Processed 2100/44441 images.
Processed 2200/44441 images.
Processed 2300/44441 images.
Processed 2400/44441 images.
Processed 2500/44441 images.
Processed 2600/44441 images.
Processed 2700/44441 images.
Processed 2800/44441 images.
Processed 2900/44441 images.
Processed 3000/44441 images.
Processed 3100/44441 images.
Processed 3200/44441 ima

In [20]:
# Load the index and metadata once
# This is done after the potential index creation step
index, image_paths = None, None # Re-initialize before loading
try:
    # Check again if index and metadata files exist after potential creation
    if os.path.exists(INDEX_FILE) and os.path.exists(METADATA_FILE):
        index = faiss.read_index(INDEX_FILE)
        with open(METADATA_FILE, 'r') as f:
            image_paths = json.load(f)
        print("FAISS index and image paths loaded successfully.")
    else:
        print(f"Error: Index files ({INDEX_FILE}, {METADATA_FILE}) still not found after attempted creation.")
        print("Please check the indexing process and ensure the image directory exists and contains images.")

except Exception as e:
    print(f"Error during initial index loading after creation attempt: {e}")
    index = None
    image_paths = None

FAISS index and image paths loaded successfully.


In [21]:
# --- Helper Functions (combined from previous cells) ---

def get_text_embedding(text_prompt: str, model, tokenizer, device) -> np.ndarray:
    """ Generates the CLIP embedding vector for a single text prompt."""
    if model is None or tokenizer is None:
        print("Error: CLIP model or tokenizer not loaded.")
        return None
    try:
        # Preprocess the text
        inputs = tokenizer(
            [text_prompt],
            padding=True,
            return_tensors="pt",
            truncation=True
        ).to(device)

        # Generate the embedding using the text encoder
        with torch.no_grad():
            text_features = model.get_text_features(**inputs)

        # Normalize the embedding (CRITICAL: must match the image indexing normalization)
        embedding = text_features.cpu().numpy().flatten()
        embedding = embedding / np.linalg.norm(embedding)

        # Ensure it's float32 for FAISS
        return embedding.astype('float32')
    except Exception as e:
        print(f"Error processing text prompt '{text_prompt}': {e}")
        return None


def search_image_similarity(query_vector: np.ndarray, index, k: int = 5):
    """ Performs the search on the FAISS index."""
    if index is None:
        print("Error: FAISS index not loaded.")
        return None, None
    try:
        query_vector = np.expand_dims(query_vector, axis=0) # Reshape for FAISS
        distances, indices = index.search(query_vector, k)
        return indices[0], distances[0]
    except Exception as e:
        print(f"Error during FAISS search: {e}")
        return None, None

API_URL = "https://api-inference.huggingface.co/models/openai/whisper-large-v3-turbo"
# headers variable is defined at the top


def transcribe_speech(audio_data):
    """
    Transcribes speech from audio data using the Hugging Face Whisper API.

    Args:
        audio_data: A tuple (sampling_rate, audio_array) where audio_array is a numpy array.

    Returns:
        A string containing the transcribed text or an error message.
    """
    if audio_data is None:
        return "Please provide audio data."

    try:
        # When type="numpy", gradio provides a tuple (sampling_rate, audio_array)
        sampling_rate, audio_array = audio_data

        # Convert numpy array to bytes in WAV format
        with io.BytesIO() as wav_buffer:
            # Ensure audio_array is float32 as required by soundfile
            if audio_array.dtype != np.float32:
                audio_array = audio_array.astype(np.float32)
            sf.write(wav_buffer, audio_array, sampling_rate, format='WAV')
            data = wav_buffer.getvalue()

        # Ensure 'headers' variable is accessible (defined at the top)
        if 'headers' not in globals():
            print("Error: Hugging Face API headers are not defined.")
            return "Error: API headers not configured."

        response = requests.post(API_URL, headers=headers, data=data)
        response.raise_for_status() # Raise an exception for bad status codes

        response_json = response.json()
        # Correctly access the text from the dictionary response
        if 'text' in response_json:
            return response_json['text'].strip() # Return only the text, stripped of leading/trailing whitespace
        else:
            print(f"API Response does not contain 'text' key: {response_json}")
            return "Error: API Response does not contain 'text' key."

    except requests.exceptions.RequestException as e:
        print(f"API Request Error: {e}")
        # We need to handle the case where 'response' might not be defined if the request itself fails
        if 'response' in locals() and response is not None:
             print(f"Response Status Code: {response.status_code}")
             print(f"Response Content: {response.text}")
        return f"API Request Error: {e}"
    except Exception as e:
        print(f"Processing Error: {type(e).__name__}: {e}") # Print the exception type
        return f"Processing Error: {type(e).__name__}: {e}"

In [22]:
# --- User Feedback ---

# Create the feedback file if it doesn't exist
if not os.path.exists("user_feedback.txt"):
    with open("user_feedback.txt", "w") as f:
        f.write("User Feedback:\n")
    print("Created user_feedback.txt")

# The submit_feedback_fn is defined below in ZLJj43QfMNYk

Created user_feedback.txt


In [23]:
from transformers import pipeline # For sentiment analysis

# ==========================================================
# 1️⃣  Initialize Sentiment Analysis Model (Hugging Face)
# ==========================================================
sentiment_pipe = pipeline(
    "text-classification",
    model="tabularisai/multilingual-sentiment-analysis",
    token=HF_API_TOKEN
)

config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [24]:
# ==========================================================
# 3️⃣  Feedback Function with Sentiment Analysis
# ==========================================================
def submit_feedback_fn(text_query, audio_query, image_query, transcribed_text_output, feedback_text, rating):
    input_type = ""
    query_value = ""

    # Determine the input type used
    if text_query and text_query.strip():
        input_type = "text"
        query_value = text_query

    if audio_query is not None:
        input_type += " speech"
        query_value += transcribed_text_output if transcribed_text_output else "N/A"

    if image_query is not None:
        input_type += " image"
        query_value += str(image_query)

    if not input_type:
        input_type = "unknown"
        query_value = "N/A"


# ==========================================================
# 🔹 Sentiment Analysis + Rating Integration (Using Hugging Face)
# ==========================================================
    try:
        sentiment_result = sentiment_pipe(feedback_text)[0]
        sentiment_label = sentiment_result["label"]
        sentiment_score = round(sentiment_result["score"], 3)

        # Combine sentiment with rating
        if rating >= 4 and sentiment_label == "NEGATIVE":
            final_sentiment = f"Conflict 🤔 (Positive rating but negative text)"
        elif rating <= 2 and sentiment_label == "POSITIVE":
            final_sentiment = f"Conflict 🤔 (Negative rating but positive text)"
        else:
            # Weighted average for consistency (0–1 scale)
            normalized_rating = rating / 5.0
            hybrid_score = round((sentiment_score + normalized_rating) / 2, 3)
            if (rating >= 4 and sentiment_label == "NEGATIVE") or (rating <= 2 and sentiment_label == "POSITIVE"):
                final_sentiment = f"Conflict 🤔 (rating={rating}, text={sentiment_label} {sentiment_score})"
            else:
                final_sentiment = f"{sentiment_label} ({hybrid_score})"

    except Exception as e:
        sentiment_label = "Error"
        sentiment_score = 0.0
        final_sentiment = f"Error (0.0)"
        print(f"Sentiment analysis failed: {e}")

    # ==========================================================
    # 🔹 Save Feedback
    # ==========================================================
    with open("user_feedback.txt", "a", encoding="utf-8") as f:
        f.write(
        f"Type: {input_type} | Query: {query_value} | Feedback: {feedback_text} "
        f"| Final Sentiment: {final_sentiment} | Raw Sentiment: {sentiment_label} ({sentiment_score}) | Rating: {rating}\n"
    )

    return f"✅ Thanks, Your Feedback has been recorded"

In [25]:

# loading  text-to-speech model
processor_tts = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model_tts = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder_tts = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/585M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

In [26]:
# generate random speaker embedding
speaker_embeddings = torch.randn(1, 512)
def text_to_speech(text):
    if not text or text.strip() == "":
        return None
    try:
        inputs = processor_tts(text=text, return_tensors="pt")
        speech = model_tts.generate_speech(
            inputs["input_ids"],
            speaker_embeddings=speaker_embeddings,
            vocoder=vocoder_tts
        )
        tmpfile = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        sf.write(tmpfile.name, speech.numpy(), 16000)
        return tmpfile.name
    except Exception as e:
        print("TTS Error:", e)
        return None

In [27]:
def multimodal_search_interface(text_input, image_input, audio_input):
    """
    Handles multimodal search queries from the Gradio interface.

    Args:
        text_input: Text entered in the text box.
        image_input: Uploaded image file path (str) or None.
        audio_input: Audio data (tuple or None) from the microphone.

    Returns:
        A tuple containing:
        - A list of tuples [(score, PIL_Image), ...] for Gradio Gallery output,
          or an empty list if an error occurred or no results were found.
        - A string containing the transcribed text if audio was used, otherwise an empty string.
    """
    # Check if essential components are loaded
    if model is None or processor is None or tokenizer is None or index is None or image_paths is None:
        error_message = "Error: Essential components (model, index, etc.) not loaded. Please check the setup steps."
        print(error_message)
        # Return empty results and the error message
        return [], error_message

    query_vector = None
    query_type = None
    query_display = "No query provided."
    search_performed = False
    transcribed_text = "" # Initialize transcribed text
    text_embedding = None
    image_embedding = None
    

    # Process audio first if provided
    if audio_input is not None:
        transcribed_text = transcribe_speech(audio_input)
        if "Error" in transcribed_text or "Please provide audio" in transcribed_text:
             print(f"Audio transcription failed: {transcribed_text}")
             return [], transcribed_text
        # Use transcribed text as the primary text input
        text_input = transcribed_text

    # Get text embedding if text input is provided (either directly or from audio)
    if text_input and text_input.strip():
        text_embedding = get_text_embedding(text_input.strip(), model, tokenizer, device)
        query_type = 'text'
        query_display = f"Text Query: '{text_input.strip()}'"
        search_performed = True

    # Get image embedding if image input is provided
    if image_input is not None:
        # image_input from gr.Image(type="filepath") is the file path
        image_embedding = get_image_embedding(image_input, model, processor, device)
        if query_type is None: # If no text query yet
            query_type = 'image'
            query_display = f"Image Query: {os.path.basename(image_input)}"
        else: # If there is also a text query
            query_type = 'text and image'
            query_display += f" + Image Query: {os.path.basename(image_input)}"
        search_performed = True

    # Combine embeddings if both are available
    if text_embedding is not None and image_embedding is not None:
        # Simple averaging of normalized embeddings
        query_vector = (text_embedding + image_embedding) / 2
        query_vector = query_vector / np.linalg.norm(query_vector) # Re-normalize
    elif text_embedding is not None:
        query_vector = text_embedding
    elif image_embedding is not None:
        query_vector = image_embedding

    if not search_performed or query_vector is None:
        info_message = "Please provide a text query, upload an image, or speak your query."
        print(info_message)
        return [], info_message

    if query_vector is None:
        error_message = f"Failed to generate embedding for the query: {query_display}"
        print(error_message)
        return [], error_message

    # Perform search
    K_RESULTS = 9 # change as output cases
    indices, scores = search_image_similarity(query_vector, index, K_RESULTS)

    if indices is None or len(indices) == 0:
        info_message = f"No similar items found for your query: {query_display}"
        print(info_message)
        return [], info_message

    # Prepare results for Gradio Gallery
    results_for_gallery = []

    for index_id, score in zip(indices, scores):
        try:
            if 0 <= index_id < len(image_paths): # Add bounds checking
                result_path = image_paths[index_id]
                if os.path.exists(result_path): # Check if file exists
                    result_image = Image.open(result_path)
                    # Gradio Gallery expects a tuple of (image, label/caption)
                    results_for_gallery.append((result_image, f"Score: {score:.4f}"))
                else:
                    print(f"Warning: Image file not found at {result_path}. Skipping.")
            else:
                 print(f"Warning: Index {index_id} out of bounds for image_paths ({len(image_paths)}). Skipping.")

        except Exception as e:
            print(f"Error processing result image with index_id {index_id}: {e}. Skipping.")
            continue

    if not results_for_gallery:
         info_message = f"No valid result images found for your query: {query_display}"
         print(info_message)
         return [], info_message
    

    return results_for_gallery, transcribed_text

In [ ]:
# --- Create and Launch Gradio Interface ---
print("Launching Gradio Interface...")

if model is not None and processor is not None and tokenizer is not None and index is not None and image_paths is not None:
    # Define the Gradio interface
    # Using gr.Row to place components on the same line
    with gr.Blocks() as iface:
        gr.Markdown(
            """
            # Multimodal Product Search
            Search for products using text, image, or speech. Ensure you have placed images in the 'data_images' directory for indexing before running the search.
            """
        )
           
        with gr.Row():
            text_input = gr.Textbox(label="Text Query (Optional)", scale=2) # Use scale to control relative width
            audio_input = gr.Audio(sources=["microphone"], type="numpy", label="Record Audio Query (Optional)", scale=1) # Use scale to control relative width

        image_input = gr.Image(type="filepath", label="Upload Image Query (Optional)", height=300)
        search_button = gr.Button("Search")
        transcribed_text_output = gr.Textbox(label="Transcribed Speech", interactive=False) # Make transcribed text output non-interactive
        gallery_output = gr.Gallery(label="Search Results", columns=3, rows=3, object_fit="contain", height=400)
        
         # Add a welcome message audio component that autoplays
        welcome_audio = text_to_speech("Welcome to the product search system.")
        if welcome_audio:
            display(ipd.Audio(welcome_audio, autoplay=True))
            
        # Link the search button click to the search function
        search_button.click(
            fn=multimodal_search_interface,
            inputs=[text_input, image_input, audio_input],
            outputs=[gallery_output, transcribed_text_output])

        # ================== 🔹 Feedback Section (Start) 🔹 ==================
        gr.Markdown("---")
        gr.Markdown("### 📝 Feedback Section")

        with gr.Row():
            feedback_text = gr.Textbox(
                label="Write your feedback here",
                placeholder="Share your thoughts about your experience...",
                lines=4,
                scale=2
            )
            rating = gr.Slider(
                minimum=1,
                maximum=5,
                step=1,
                label="Rating (1–5 ⭐)",
                value=5,
                scale=1
            )

        submit_feedback = gr.Button("Submit Feedback")
        feedback_output = gr.Textbox(label="Status", interactive=False)


        submit_feedback.click(
            fn=submit_feedback_fn,
            inputs=[text_input, audio_input, image_input, transcribed_text_output, feedback_text, rating], # Using transcribed text as a proxy for the query
            outputs=[feedback_output]
        )

        # ================== 🔹 Feedback Section (End) 🔹 ==================

    # Launch the interface
    iface.launch(share=True, debug=True)
else:
    print("Cannot launch Gradio interface. Essential components are not loaded. Please check the setup steps.")

Launching Gradio Interface...


model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8b59b0bf6510a58a64.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
